<a href="https://colab.research.google.com/github/digyamandala/densenet121-transfer-learning/blob/master/densenet121_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import tensorflow as tf
import numpy as np
from pathlib import Path

In [2]:
from tensorflow.keras.applications import DenseNet121

In [25]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [89]:
for p in Path("*").glob("*"):
  print(p)

In [134]:
ROOT = "/content/gdrive/My Drive" 

TEST_PATH = ROOT + "/Covid19-dataset/test"

TRAIN_PATH = ROOT + "/Covid19-dataset/train"

TRAIN_COVID = TRAIN_PATH + "/Covid/"
TRAIN_NORMAL = TRAIN_PATH + "/Normal/"
TRAIN_VIRAL_PNEUMONIA = TRAIN_PATH + "/Viral Pneumonia/"

TEST_COVID = TEST_PATH + "/Covid/"
TEST_NORMAL = TEST_PATH + "/Normal/"
TEST_VIRAL_PNEUMONIA = TEST_PATH + "/Viral Pneumonia/"

In [135]:
train_covid_path = Path(TRAIN_COVID)
train_normal_path = Path(TRAIN_NORMAL)
train_viral_pneumonia_path = Path(TRAIN_VIRAL_PNEUMONIA)

test_covid_path = Path(TEST_COVID)
test_normal_path = Path(TEST_NORMAL)
test_viral_pneumonia_path = Path(TEST_VIRAL_PNEUMONIA)

In [121]:
def loadImage(home_directory: Path, label: int):
  x_train = []
  y_train = []
  for path in home_directory.glob("*.*"):
    image = tf.keras.preprocessing.image.load_img(path, target_size=(128,128))
    np_image = np.array(image)
    x_train.append(np_image)
    y_train.append(np.array([label]))
  return np.array(x_train), np.array(y_train)


In [122]:
x_train_covid, y_train_covid = loadImage(train_covid_path, 0)
x_train_normal, y_train_normal = loadImage(train_normal_path, 1)
x_train_viral, y_train_viral = loadImage(train_viral_pneumonia_path, 2)

In [136]:
x_test_covid, y_test_covid = loadImage(test_covid_path, 0)
x_test_normal, y_test_normal = loadImage(test_normal_path, 1)
x_test_viral, y_test_viral = loadImage(test_viral_pneumonia_path, 2)

In [125]:
x_train = np.concatenate((x_train_covid, x_train_normal, x_train_viral))
y_train = np.concatenate((y_train_covid, y_train_normal, y_train_viral))

In [137]:
x_test = np.concatenate((x_test_covid, x_test_normal, x_test_viral))
y_test = np.concatenate((y_test_covid, y_test_normal, y_test_viral))

In [145]:
x_train_normalized = x_train / 255
x_test_normalized = x_test / 255
y_train_hot_vector = tf.keras.utils.to_categorical(y_train, 3)
y_test_hot_vector = tf.keras.utils.to_categorical(y_test, 3)

In [177]:
denseNet = DenseNet121(include_top=False, weights="imagenet", input_shape=(128,128,3))

In [178]:
model = tf.keras.Sequential()
model.add(denseNet)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(3, activation="softmax"))
model.summary()
model.compile(optimizer="adam", loss=tf.keras.losses.categorical_crossentropy, metrics=["accuracy"])

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 4, 4, 1024)        7037504   
_________________________________________________________________
flatten_8 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_11 (Dense)             (None, 3)                 49155     
Total params: 7,086,659
Trainable params: 7,003,011
Non-trainable params: 83,648
_________________________________________________________________


In [179]:
model.fit(
    x=x_train_normalized, 
    y=y_train_hot_vector, 
    batch_size=2, 
    validation_split=0.1, 
    epochs=10, 
    shuffle=True, )

Epoch 1/10
113/113 [==============================] - 12s 102ms/step - loss: 3.7898 - accuracy: 0.6444 - val_loss: 61.5125 - val_accuracy: 0.0000e+00
Epoch 2/10
113/113 [==============================] - 9s 83ms/step - loss: 3.8221 - accuracy: 0.6667 - val_loss: 71.3709 - val_accuracy: 0.3846
Epoch 3/10
113/113 [==============================] - 9s 83ms/step - loss: 4.6305 - accuracy: 0.7022 - val_loss: 21.6253 - val_accuracy: 0.1154
Epoch 4/10
113/113 [==============================] - 9s 84ms/step - loss: 2.8779 - accuracy: 0.7867 - val_loss: 1.2088 - val_accuracy: 0.9231
Epoch 5/10
113/113 [==============================] - 9s 84ms/step - loss: 2.0992 - accuracy: 0.7911 - val_loss: 6.2605 - val_accuracy: 0.5769
Epoch 6/10
113/113 [==============================] - 9s 83ms/step - loss: 1.5771 - accuracy: 0.8267 - val_loss: 4.2754 - val_accuracy: 0.6154
Epoch 7/10
113/113 [==============================] - 9s 83ms/step - loss: 1.2670 - accuracy: 0.8444 - val_loss: 3.9230 - val_accurac

In [181]:
model.evaluate(x_test_normalized, y_test_hot_vector)

3/3 [==============================] - 0s 49ms/step - loss: 2.8688 - accuracy: 0.8030


[2.8687891960144043, 0.8030303120613098]

In [198]:
x_test_normal.shape

(20, 128, 128, 3)

In [213]:
np.argmax(model.predict(np.array([x_test_normalized[26 + 26]])))

2